# Convolutional Generative Adversarial Network (DCGAN) - PyTorch
This notebook contains the same implementation of a Convolutional Generative Adversarial Network (DCGAN) as the python files. The notebook is primarily meant to give a better understanding of the model without the pain to search through the python files. In addition to that code for a run in collaboratory is provided.

GANs were first [introduced](https://arxiv.org/abs/1406.2661) in 2014 from Ian Goodfellow and others. A [DCGAN](https://arxiv.org/abs/1511.06434) is an adaption of the normal GAN in which all linear layers are replaced with convolutional layers for better performance. GANs consists of two networks: a generator ***G*** and a discriminator ***D***. The generator is designed to generate realistic looking data "fake data" and the discriminator is designed to distinguish between generated "fake data" and real data from a dataset. Both networks play a game against each other where the generator tries to fool the discriminator by generating data as real as possible and the discriminator tries to classify fake and real data correctly. 

The generator is a convolutional network which takes a random vector as input and outputs another vector which can be reshaped to an image
The discriminator is a convolutional classifier that takes an image vector as input and outputs a probability of that image being real.
This game leads to a generator producing data that is indistinguishable from real data to the discriminator

## Google Collaboratory Code
Uncommend the following cells if you're running this notebook in google collab.

In [ ]:
#-----MOUNT DRIVE-----
#from google.colab import drive
#drive.mount('/gdrive')

In [ ]:
#-----INSTALL PYTORCH-----
#from os.path import exists
#from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
#platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
#cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
#accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [ ]:
#-----REINSTALL PILLOW ON COLLAB-----
#!pip install Pillow==4.0.0
#!pip install PIL
#!pip install image

## Import Libraries

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Import PyTorch dependencies
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

%matplotlib inline

In [154]:
# Make directories for images and svhn dataset
os.makedirs('svhn_data', exist_ok=True)
os.makedirs('generated_images', exist_ok=True)

## Import and Transform Dataset

The [SVHN](http://ufldl.stanford.edu/housenumbers/) dataset will be used as an example dataset if nothing else is specified. For a custom dataset, change the custom_image_path parameter. JPEG images are recommended.

In [155]:
#-----DATASET PARAMETERS-----
custom_image_path = None
batch_size = 128
# Don't change this parameter unless you are using a different kernel
# No linear layers are used so the dimensions after the last convolition need to be 1*1*1
image_size = 65
#---------------------------

# Resize data
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor()
])

if custom_image_path != None:
    train_dataset = datasets.ImageFolder(custom_image_path)
else:
    train_dataset = datasets.SVHN(root="svhn_data", split="train", download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)

Using downloaded and verified file: svhn_data/train_32x32.mat


In [ ]:
def im_convert(tensor, rescale=False):
    # Clone image and transfer it to CPU
    image = tensor.cpu().detach()
    image = image.numpy().squeeze()
    # Switch dimensions form (3, 200, 200) to (200, 200, 3)
    image = np.transpose(image, (1, 2, 0))
    if rescale:
        # Rescale image from tanh output (1, -1) to rgb (0, 255)
        image = ((image + 1) * 255 / (2)).astype(np.uint8)
    return image

# Show one image 
dataiter = iter(train_loader)
images, labels = dataiter.next()
image = im_convert(images[0])
plt.imshow(image)

In [ ]:
# Scale image values from -1 to 1 to be close to the output of the tanh function
def scale(x, feature_range=(-1, 1)):
    min, max = feature_range
    x = x * (max-min) + min
    return x

## GAN Models

### Discriminator

In [ ]:
# Function to create convolutional layers with batch normalization and without bias terms
def conv(in_channels, out_channels, kernel_size, stride=1, padding=0, batch_norm=True):
    layers = []
    if batch_norm:
        # If batch_norm is true add a batch norm layer
        conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=False)
        batch_norm = nn.BatchNorm2d(out_channels)
        layers = [conv_layer, batch_norm]
    else:
        # If batch_norm is false just add a conv layer
        conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=False)
        layers.append(conv_layer)
    return nn.Sequential(*layers)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, conv_dim=32):
        super().__init__()
        # Define hidden convolutional layers
        self.input = conv(3, conv_dim, kernel_size=5, stride=2, padding=2, batch_norm=False)
        self.conv1 = conv(conv_dim, conv_dim*2, kernel_size=5, stride=2, padding=2)
        self.conv2 = conv(conv_dim*2, conv_dim*4, kernel_size=5, stride=2, padding=2)
        self.conv3 = conv(conv_dim*4, conv_dim*8, kernel_size=5, stride=2, padding=2)
        self.output = conv(conv_dim*8, 1, kernel_size=5, stride=1, padding=0, batch_norm=False)
        # Activation function
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.02)
    def forward(self, x):
        x = self.leaky_relu(self.input(x))
        x = self.leaky_relu(self.conv1(x))
        x = self.leaky_relu(self.conv2(x))
        x = self.leaky_relu(self.conv3(x))
        x = torch.sigmoid(self.output(x))
        return x

### Generator

In [ ]:
# Function to create transpose convolutional layers with batch normalization and without bias terms
def conv_trans(in_channels, out_channels, kernel_size, stride=1, padding=0, batch_norm=True):
    layers = []
    if batch_norm:
        # If batch_norm is true add a batch norm layer
        conv_layer = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=False)
        batch_norm = nn.BatchNorm2d(out_channels)
        layers = [conv_layer, batch_norm]
    else:
        # If batch_norm is false just add a transpose conv layer
        conv_layer = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=False)
        layers.append(conv_layer)
    return nn.Sequential(*layers)

In [ ]:
class Generator(nn.Module):
    def __init__(self, conv_dim=32, z_dim=100):
        super().__init__()
        
        # Define hidden transpose convolutional layers
        self.input = conv_trans(z_dim, conv_dim*8, kernel_size=5, stride=1, padding=0)
        self.conv_trans1 = conv_trans(conv_dim*8, conv_dim*4, kernel_size=5, stride=2, padding=2)
        self.conv_trans2 = conv_trans(conv_dim*4, conv_dim*2, kernel_size=5, stride=2, padding=2)
        self.conv_trans3 = conv_trans(conv_dim*2, conv_dim, kernel_size=5, stride=2, padding=2)
        self.output = conv_trans(conv_dim, 3, kernel_size=5, stride=2, padding=2, batch_norm=False)
    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.conv_trans1(x))
        x = F.relu(self.conv_trans2(x))
        x = F.relu(self.conv_trans3(x))
        x = torch.tanh(self.output(x))
        return x

### Instantiate Models

In [ ]:
# Build Network
z_dim = 100
conv_dim = 32

D = Discriminator(conv_dim=conv_dim)
G = Generator(conv_dim=conv_dim, z_dim=z_dim)

In [ ]:
# If a gpu is available move all models to gpu
if torch.cuda.is_available():
    G = G.cuda()
    D = D.cuda()
    print("GPU available. Moved models to GPU.")
else:
    print("Training on CPU.")

In [ ]:
def real_loss(predictions, smooth=False):
    batch_size = predictions.shape[0]
    labels = torch.ones(batch_size)
    # Smooth labels for discriminator to weaken learning
    if smooth:
        labels = labels * 0.9
    # We use the binary cross entropy loss | Model has a sigmoid function
    criterion = nn.BCELoss()
    # Move models to GPU if available
    if torch.cuda.is_available():
        labels = labels.cuda()
        criterion = criterion.cuda()
    loss = criterion(predictions.squeeze(), labels)
    return loss

def fake_loss(predictions):
    batch_size = predictions.shape[0]
    labels = torch.zeros(batch_size)
    criterion = nn.BCELoss()
    # Move models to GPU if available
    if torch.cuda.is_available():
        labels = labels.cuda()
        criterion = criterion.cuda()
    loss = criterion(predictions.squeeze(), labels)
    return loss

In [ ]:
def random_vector(batch_size, length):
    # Sample from a Gaussian distribution
    z_vec = torch.randn(batch_size, length, 1, 1).float()
    if torch.cuda.is_available():
        z_vec = z_vec.cuda()
    return z_vec

## Training

### Optimizer

In [ ]:
#-----TRAINING PARAMETERS-----
lr = 0.0002
beta1 = 0.5
beta2 = 0.999
#-----------------------------

# Adam optimizer as trainigs function
d_optimizer = torch.optim.Adam(D.parameters(), lr=lr, betas=[beta1, beta2])
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr, betas=[beta1, beta2])

In [ ]:
def train_discriminator(generator, discriminator, optimizer, real_data, batch_size, z_size):
    # Set discriminator into training mode and reset gradients
    discriminator.train()
    optimizer.zero_grad()
    # Rescale images into -1 to 1 range
    real_data = scale(real_data)
    # Train on real data
    real_data_logits = discriminator.forward(real_data)
    loss_real = real_loss(real_data_logits, smooth=True)
    # Train on fake data
    z_vec = random_vector(batch_size, z_size)
    fake_data = generator.forward(z_vec)
    fake_data_logits = discriminator.forward(fake_data)
    loss_fake = fake_loss(fake_data_logits)
    # Calculate total loss, gradients and take optimization step
    total_loss = loss_fake + loss_real
    total_loss.backward()
    optimizer.step()
    return total_loss

In [ ]:
def train_generator(generator, discriminator, optimizer, batch_size, z_size):
    # Reset gradients and set model to training mode
    generator.train()
    optimizer.zero_grad()
    # Generate fake data
    z_vec = random_vector(batch_size, z_size)
    fake_data = generator.forward(z_vec)
    # Train generator with output of discriminator
    discriminator_logits = discriminator.forward(fake_data)
    # Reverse labels
    loss = real_loss(discriminator_logits)
    loss.backward()
    optimizer.step()
    return loss

### Trainigs loop

In [ ]:
epochs = 100
# After how many batches should generated sample images be shown?
print_every = 200
# How many images should be shown?
sample_size = 8
# After how many epochs should the loss be plotted?
plot_every = 5
# Create some sample noise
sample_noise = random_vector(sample_size, z_dim)
#-------------------------

# Keep track of losses
d_losses = []
g_losses = []

for e in range(epochs):
    for batch_i, (images, _) in enumerate(train_loader):
        batch_size = images.shape[0]
        # Move images to GPU if available
        if torch.cuda.is_available():
            images = images.cuda()
        # Train discriminator
        d_loss = train_discriminator(G, D, d_optimizer, images, batch_size, z_dim)
        # Train generator
        g_loss = train_generator(G, D, g_optimizer, batch_size, z_dim)
        
        # Keep track of losses
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        
        # Print some sample pictures
        if (batch_i % print_every == 0):
            print("Epoch: {}, Batch: {}, D-Loss: {}, G-Loss: {}".format(e, batch_i, d_loss, g_loss))
            # Make sample generation
            G.eval()
            fig, axes = plt.subplots(1, sample_size, figsize=(20, 10))
            # Generate predictions
            predictions = G.forward(sample_noise)
            for i in range(sample_size):
                axes[i].imshow(im_convert(predictions[i], rescale=True))
            plt.show()
    if (e % plot_every == 0):
        # Print losses
        plt.plot(d_losses, label="Discriminator", alpha=0.5)
        plt.plot(g_losses, label="Generator", alpha=0.5)
        plt.title("Trainings loss")
        plt.legend()
        plt.show()

## Validation

In [ ]:
plt.plot(d_losses, label="Discriminator", alpha=0.5)
plt.plot(g_losses, label="Generator", alpha=0.5)
plt.title("Trainings loss")
plt.legend()
plt.show()

In [ ]:
def show_samples(num_samples):
    G.eval()
    z_vec = random_vector(num_samples, z_dim)
    predictions = G.forward(z_vec)
    fig, axes = plt.subplots(1, num_samples, figsize=(20, 10))
    for i in range(num_samples):
        axes[i].imshow(im_convert(predictions[i], rescale=True), cmap="gray")              
    plt.show()

In [ ]:
show_samples(8)